In [0]:
applications_df = (
    spark.read
    .option("header", "true")
    .csv("/Volumes/workspace/default/rawdata/steam/bi_applications_capped.csv")
)

reviews_app_df = (
    spark.read
    .option("header", "true")
    .csv("/Volumes/workspace/default/rawdata/steam/app_review_agg.csv")
)


In [0]:
# Schema check
applications_df.printSchema()


# Row counts
print("Applications rows:", applications_df.count())


# Sample rows
display(applications_df.limit(5))



root
 |-- appid: string (nullable = true)
 |-- name: string (nullable = true)
 |-- type: string (nullable = true)
 |-- is_free: string (nullable = true)
 |-- mat_initial_price: string (nullable = true)
 |-- mat_final_price: string (nullable = true)
 |-- mat_discount_percent: string (nullable = true)
 |-- mat_currency: string (nullable = true)
 |-- metacritic_score: string (nullable = true)
 |-- recommendations_total: string (nullable = true)
 |-- mat_achievement_count: string (nullable = true)
 |-- mat_supports_windows: string (nullable = true)
 |-- mat_supports_mac: string (nullable = true)
 |-- mat_supports_linux: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- required_age: string (nullable = true)
 |-- mat_initial_price_capped: string (nullable = true)
 |-- mat_final_price_capped: string (nullable = true)
 |-- recommendations_total_capped: string (nullable = true)
 |-- mat_achievement_count_capped: string (nullable = true)

Applications rows: 239664


appid,name,type,is_free,mat_initial_price,mat_final_price,mat_discount_percent,mat_currency,metacritic_score,recommendations_total,mat_achievement_count,mat_supports_windows,mat_supports_mac,mat_supports_linux,release_date,required_age,mat_initial_price_capped,mat_final_price_capped,recommendations_total_capped,mat_achievement_count_capped
10,Counter-Strike,game,false,999.0,999.0,0.0,USD,88.0,161854,null,true,true,true,2000-11-01,0,999.0,999.0,76489,null
20,Team Fortress Classic,game,false,499.0,499.0,0.0,USD,null,6633,null,true,true,true,1999-04-01,0,499.0,499.0,6633,null
30,Day of Defeat,game,false,499.0,499.0,0.0,USD,79.0,4308,null,true,true,true,2003-05-01,0,499.0,499.0,4308,null
40,Deathmatch Classic,game,false,499.0,499.0,0.0,USD,null,2338,null,true,true,true,2001-06-01,0,499.0,499.0,2338,null
50,Half-Life: Opposing Force,game,false,499.0,499.0,0.0,USD,null,22793,null,true,true,true,1999-11-01,0,499.0,499.0,22793,null


In [0]:
reviews_app_df.printSchema()
print("Reviews (app-level) rows:", reviews_app_df.count())
display(reviews_app_df.limit(5))

root
 |-- appid: string (nullable = true)
 |-- total_reviews: string (nullable = true)
 |-- avg_votes_up: string (nullable = true)
 |-- avg_votes_funny: string (nullable = true)
 |-- avg_comment_count: string (nullable = true)
 |-- avg_weighted_vote_score: string (nullable = true)
 |-- avg_playtime_forever: string (nullable = true)
 |-- pct_steam_purchase: string (nullable = true)
 |-- pct_received_for_free: string (nullable = true)
 |-- pct_early_access: string (nullable = true)

Reviews (app-level) rows: 117311


appid,total_reviews,avg_votes_up,avg_votes_funny,avg_comment_count,avg_weighted_vote_score,avg_playtime_forever,pct_steam_purchase,pct_received_for_free,pct_early_access
1362650,17,1.1764705882352942,0.0,0.058823529411764705,0.5073456358823529,149.1764705882353,1.0,0.0,0.0
525640,100,5.26,0.23,0.22,0.5514197209000001,2874.67,1.0,0.01,0.74
962300,4,0.0,0.0,0.0,0.4953703725,0.0,1.0,0.0,0.0
617580,9,2.0,0.0,0.1111111111111111,0.5171290133333334,3744.6666666666665,1.0,0.1111111111111111,0.0
1633470,4,2.75,0.25,0.5,0.553060665,60.5,1.0,0.25,0.0


In [0]:
master_step1_df = (
    applications_df
    .join(
        reviews_app_df,
        on="appid",
        how="left"
    )
)


In [0]:
# Row count check (CRITICAL)
print("Rows after join:", master_step1_df.count())

# Check for duplication of appid
from pyspark.sql.functions import countDistinct, count

master_step1_df.select(
    countDistinct("appid").alias("distinct_appids"),
    count("*").alias("total_rows")
).show()

# Sample rows
display(master_step1_df.limit(5))


Rows after join: 239664
+---------------+----------+
|distinct_appids|total_rows|
+---------------+----------+
|         239664|    239664|
+---------------+----------+



appid,name,type,is_free,mat_initial_price,mat_final_price,mat_discount_percent,mat_currency,metacritic_score,recommendations_total,mat_achievement_count,mat_supports_windows,mat_supports_mac,mat_supports_linux,release_date,required_age,mat_initial_price_capped,mat_final_price_capped,recommendations_total_capped,mat_achievement_count_capped,total_reviews,avg_votes_up,avg_votes_funny,avg_comment_count,avg_weighted_vote_score,avg_playtime_forever,pct_steam_purchase,pct_received_for_free,pct_early_access
10,Counter-Strike,game,false,999.0,999.0,0.0,USD,88.0,161854,null,true,true,true,2000-11-01,0,999.0,999.0,76489,null,null,null,null,null,null,null,null,null,null
20,Team Fortress Classic,game,false,499.0,499.0,0.0,USD,null,6633,null,true,true,true,1999-04-01,0,499.0,499.0,6633,null,null,null,null,null,null,null,null,null,null
30,Day of Defeat,game,false,499.0,499.0,0.0,USD,79.0,4308,null,true,true,true,2003-05-01,0,499.0,499.0,4308,null,null,null,null,null,null,null,null,null,null
40,Deathmatch Classic,game,false,499.0,499.0,0.0,USD,null,2338,null,true,true,true,2001-06-01,0,499.0,499.0,2338,null,null,null,null,null,null,null,null,null,null
50,Half-Life: Opposing Force,game,false,499.0,499.0,0.0,USD,null,22793,null,true,true,true,1999-11-01,0,499.0,499.0,22793,null,null,null,null,null,null,null,null,null,null


In [0]:
app_developers_agg_df = (
    spark.read.option("header", "true")
    .csv("/Volumes/workspace/default/rawdata/steam/processed/aggregates/app_developers/")
)

app_publishers_agg_df = (
    spark.read.option("header", "true")
    .csv("/Volumes/workspace/default/rawdata/steam/processed/aggregates/app_publishers/")
)

app_genres_agg_df = (
    spark.read.option("header", "true")
    .csv("/Volumes/workspace/default/rawdata/steam/processed/aggregates/app_genres/")
)

app_categories_agg_df = (
    spark.read.option("header", "true")
    .csv("/Volumes/workspace/default/rawdata/steam/processed/aggregates/app_categories/")
)

app_platforms_agg_df = (
    spark.read.option("header", "true")
    .csv("/Volumes/workspace/default/rawdata/steam/processed/aggregates/app_platforms/")
)


In [0]:
# Row counts
print("Developers agg rows:", app_developers_agg_df.count())
print("Publishers agg rows:", app_publishers_agg_df.count())
print("Genres agg rows:", app_genres_agg_df.count())
print("Categories agg rows:", app_categories_agg_df.count())
print("Platforms agg rows:", app_platforms_agg_df.count())

Developers agg rows: 228980
Publishers agg rows: 214372
Genres agg rows: 207942
Categories agg rows: 225663
Platforms agg rows: 239664


In [0]:
app_developers_agg_df.printSchema()
app_publishers_agg_df.printSchema()
app_genres_agg_df.printSchema()
app_categories_agg_df.printSchema()
app_platforms_agg_df.printSchema()

root
 |-- appid: string (nullable = true)
 |-- developers: string (nullable = true)

root
 |-- appid: string (nullable = true)
 |-- publishers: string (nullable = true)

root
 |-- appid: string (nullable = true)
 |-- genres: string (nullable = true)

root
 |-- appid: string (nullable = true)
 |-- categories: string (nullable = true)

root
 |-- appid: string (nullable = true)
 |-- platforms: string (nullable = true)



In [0]:
display(app_developers_agg_df.limit(5))
display(app_platforms_agg_df.limit(5))

appid,developers
1000010,NEXT Studios
1000080,IndieLeague Studio
1000160,SinVR
1000210,SinVR
1000280,Villain Role


appid,platforms
1000010,windows
1000030,"windows, mac"
1000050,windows
1000060,"mac, linux, windows"
1000080,"windows, mac"


In [0]:
master_step2_df = (
    master_step1_df
    .join(
        app_developers_agg_df,
        on="appid",
        how="left"
    )
)


In [0]:
# Row count must stay the same
print("Rows after developers join:", master_step2_df.count())

# Uniqueness check
from pyspark.sql.functions import countDistinct

master_step2_df.select(
    countDistinct("appid").alias("distinct_appids"),
    count("*").alias("total_rows")
).show()

# Sample check
display(master_step2_df.select("appid", "name", "developers").limit(10))


Rows after developers join: 239664
+---------------+----------+
|distinct_appids|total_rows|
+---------------+----------+
|         239664|    239664|
+---------------+----------+



appid,name,developers
10,Counter-Strike,Valve
20,Team Fortress Classic,Valve
30,Day of Defeat,Valve
40,Deathmatch Classic,Valve
50,Half-Life: Opposing Force,Gearbox Software
60,Ricochet,Valve
70,Half-Life,Valve
80,Counter-Strike: Condition Zero,Valve
130,Half-Life: Blue Shift,Gearbox Software
219,Half-Life 2: Demo,Valve


In [0]:
master_step3_df = (
    master_step2_df
    .join(
        app_publishers_agg_df,
        on="appid",
        how="left"
    )
)


In [0]:
# Row count check
print("Rows after publishers join:", master_step3_df.count())

# Uniqueness check
from pyspark.sql.functions import countDistinct

master_step3_df.select(
    countDistinct("appid").alias("distinct_appids"),
    count("*").alias("total_rows")
).show()

# Sample validation
display(
    master_step3_df
    .select("appid", "name", "publishers")
    .limit(10)
)


Rows after publishers join: 239664
+---------------+----------+
|distinct_appids|total_rows|
+---------------+----------+
|         239664|    239664|
+---------------+----------+



appid,name,publishers
10,Counter-Strike,Valve
20,Team Fortress Classic,Valve
30,Day of Defeat,Valve
40,Deathmatch Classic,Valve
50,Half-Life: Opposing Force,Valve
60,Ricochet,Valve
70,Half-Life,Valve
80,Counter-Strike: Condition Zero,Valve
130,Half-Life: Blue Shift,Valve
219,Half-Life 2: Demo,null


In [0]:
master_step4_df = (
    master_step3_df
    .join(
        app_genres_agg_df,
        on="appid",
        how="left"
    )
)


In [0]:
# Row count check
print("Rows after genres join:", master_step4_df.count())

# Uniqueness check
from pyspark.sql.functions import countDistinct

master_step4_df.select(
    countDistinct("appid").alias("distinct_appids"),
    count("*").alias("total_rows")
).show()

# Sample validation
display(
    master_step4_df
    .select("appid", "name", "genres")
    .limit(10)
)


Rows after genres join: 239664
+---------------+----------+
|distinct_appids|total_rows|
+---------------+----------+
|         239664|    239664|
+---------------+----------+



appid,name,genres
10,Counter-Strike,Action
20,Team Fortress Classic,Action
30,Day of Defeat,Action
40,Deathmatch Classic,Action
50,Half-Life: Opposing Force,Action
60,Ricochet,Action
70,Half-Life,Action
80,Counter-Strike: Condition Zero,Action
130,Half-Life: Blue Shift,Action
219,Half-Life 2: Demo,Action


In [0]:
master_step5_df = (
    master_step4_df
    .join(
        app_categories_agg_df,
        on="appid",
        how="left"
    )
)


In [0]:
# Row count check
print("Rows after categories join:", master_step5_df.count())

# Uniqueness check
from pyspark.sql.functions import countDistinct

master_step5_df.select(
    countDistinct("appid").alias("distinct_appids"),
    count("*").alias("total_rows")
).show()

# Sample validation
display(
    master_step5_df
    .select("appid", "name", "categories")
    .limit(10)
)


Rows after categories join: 239664
+---------------+----------+
|distinct_appids|total_rows|
+---------------+----------+
|         239664|    239664|
+---------------+----------+



appid,name,categories
10,Counter-Strike,"Shared/Split Screen PvP, Stereo Sound, Keyboard Only Option, Valve Anti-Cheat enabled, Color Alternatives, Multi-player, Custom Volume Controls, PvP, Online PvP, Family Sharing"
20,Team Fortress Classic,"Shared/Split Screen PvP, Stereo Sound, Keyboard Only Option, PvP, Remote Play Together, Online PvP, Multi-player, Valve Anti-Cheat enabled, Family Sharing, Custom Volume Controls"
30,Day of Defeat,"Stereo Sound, Family Sharing, Valve Anti-Cheat enabled, Color Alternatives, Multi-player, Custom Volume Controls, Camera Comfort"
40,Deathmatch Classic,"Valve Anti-Cheat enabled, Online PvP, Remote Play Together, Stereo Sound, PvP, Multi-player, Keyboard Only Option, Family Sharing, Custom Volume Controls, Color Alternatives, Shared/Split Screen PvP"
50,Half-Life: Opposing Force,"Valve Anti-Cheat enabled, Single-player, Adjustable Difficulty, Custom Volume Controls, Save Anytime, Playable without Timed Input, Family Sharing, Multi-player"
60,Ricochet,"Stereo Sound, Multi-player, Keyboard Only Option, Custom Volume Controls, Online PvP, Valve Anti-Cheat enabled, PvP, Family Sharing"
70,Half-Life,"Valve Anti-Cheat enabled, Adjustable Difficulty, Color Alternatives, Single-player, Remote Play on Phone, Custom Volume Controls, Multi-player, Keyboard Only Option, Family Sharing, Stereo Sound, PvP, Full controller support, Steam Cloud, Remote Play on Tablet, Online PvP, Save Anytime, Playable without Timed Input"
80,Counter-Strike: Condition Zero,"Valve Anti-Cheat enabled, Family Sharing, Single-player, Custom Volume Controls, Stereo Sound, Color Alternatives, Adjustable Difficulty, Multi-player"
130,Half-Life: Blue Shift,"Single-player, Save Anytime, Custom Volume Controls, Family Sharing, Adjustable Difficulty, Playable without Timed Input"
219,Half-Life 2: Demo,"Single-player, Game demo"


In [0]:
masterdata_df = (
    master_step5_df
    .join(
        app_platforms_agg_df,
        on="appid",
        how="left"
    )
)


In [0]:
# Row count check (must remain unchanged)
print("Rows after platforms join:", masterdata_df.count())

# Uniqueness check (MOST IMPORTANT)
from pyspark.sql.functions import countDistinct

masterdata_df.select(
    countDistinct("appid").alias("distinct_appids"),
    count("*").alias("total_rows")
).show()

# Sample rows (wide view)
display(
    masterdata_df.select(
        "appid", "name", "developers", "publishers",
        "genres", "categories", "platforms"
    ).limit(10)
)


Rows after platforms join: 239664
+---------------+----------+
|distinct_appids|total_rows|
+---------------+----------+
|         239664|    239664|
+---------------+----------+



appid,name,developers,publishers,genres,categories,platforms
10,Counter-Strike,Valve,Valve,Action,"Shared/Split Screen PvP, Stereo Sound, Keyboard Only Option, Valve Anti-Cheat enabled, Color Alternatives, Multi-player, Custom Volume Controls, PvP, Online PvP, Family Sharing","windows, linux, mac"
20,Team Fortress Classic,Valve,Valve,Action,"Shared/Split Screen PvP, Stereo Sound, Keyboard Only Option, PvP, Remote Play Together, Online PvP, Multi-player, Valve Anti-Cheat enabled, Family Sharing, Custom Volume Controls","linux, windows, mac"
30,Day of Defeat,Valve,Valve,Action,"Stereo Sound, Family Sharing, Valve Anti-Cheat enabled, Color Alternatives, Multi-player, Custom Volume Controls, Camera Comfort","linux, windows, mac"
40,Deathmatch Classic,Valve,Valve,Action,"Valve Anti-Cheat enabled, Online PvP, Remote Play Together, Stereo Sound, PvP, Multi-player, Keyboard Only Option, Family Sharing, Custom Volume Controls, Color Alternatives, Shared/Split Screen PvP","linux, windows, mac"
50,Half-Life: Opposing Force,Gearbox Software,Valve,Action,"Valve Anti-Cheat enabled, Single-player, Adjustable Difficulty, Custom Volume Controls, Save Anytime, Playable without Timed Input, Family Sharing, Multi-player","windows, linux, mac"
60,Ricochet,Valve,Valve,Action,"Stereo Sound, Multi-player, Keyboard Only Option, Custom Volume Controls, Online PvP, Valve Anti-Cheat enabled, PvP, Family Sharing","mac, linux, windows"
70,Half-Life,Valve,Valve,Action,"Valve Anti-Cheat enabled, Adjustable Difficulty, Color Alternatives, Single-player, Remote Play on Phone, Custom Volume Controls, Multi-player, Keyboard Only Option, Family Sharing, Stereo Sound, PvP, Full controller support, Steam Cloud, Remote Play on Tablet, Online PvP, Save Anytime, Playable without Timed Input","linux, windows, mac"
80,Counter-Strike: Condition Zero,Valve,Valve,Action,"Valve Anti-Cheat enabled, Family Sharing, Single-player, Custom Volume Controls, Stereo Sound, Color Alternatives, Adjustable Difficulty, Multi-player","mac, windows, linux"
130,Half-Life: Blue Shift,Gearbox Software,Valve,Action,"Single-player, Save Anytime, Custom Volume Controls, Family Sharing, Adjustable Difficulty, Playable without Timed Input","windows, mac, linux"
219,Half-Life 2: Demo,Valve,null,Action,"Single-player, Game demo","windows, mac"


In [0]:
display(masterdata_df.limit(5))

appid,name,type,is_free,mat_initial_price,mat_final_price,mat_discount_percent,mat_currency,metacritic_score,recommendations_total,mat_achievement_count,mat_supports_windows,mat_supports_mac,mat_supports_linux,release_date,required_age,mat_initial_price_capped,mat_final_price_capped,recommendations_total_capped,mat_achievement_count_capped,total_reviews,avg_votes_up,avg_votes_funny,avg_comment_count,avg_weighted_vote_score,avg_playtime_forever,pct_steam_purchase,pct_received_for_free,pct_early_access,developers,publishers,genres,categories,platforms
10,Counter-Strike,game,false,999.0,999.0,0.0,USD,88.0,161854,null,true,true,true,2000-11-01,0,999.0,999.0,76489,null,null,null,null,null,null,null,null,null,null,Valve,Valve,Action,"Shared/Split Screen PvP, Stereo Sound, Keyboard Only Option, Valve Anti-Cheat enabled, Color Alternatives, Multi-player, Custom Volume Controls, PvP, Online PvP, Family Sharing","windows, linux, mac"
20,Team Fortress Classic,game,false,499.0,499.0,0.0,USD,null,6633,null,true,true,true,1999-04-01,0,499.0,499.0,6633,null,null,null,null,null,null,null,null,null,null,Valve,Valve,Action,"Shared/Split Screen PvP, Stereo Sound, Keyboard Only Option, PvP, Remote Play Together, Online PvP, Multi-player, Valve Anti-Cheat enabled, Family Sharing, Custom Volume Controls","linux, windows, mac"
30,Day of Defeat,game,false,499.0,499.0,0.0,USD,79.0,4308,null,true,true,true,2003-05-01,0,499.0,499.0,4308,null,null,null,null,null,null,null,null,null,null,Valve,Valve,Action,"Stereo Sound, Family Sharing, Valve Anti-Cheat enabled, Color Alternatives, Multi-player, Custom Volume Controls, Camera Comfort","linux, windows, mac"
40,Deathmatch Classic,game,false,499.0,499.0,0.0,USD,null,2338,null,true,true,true,2001-06-01,0,499.0,499.0,2338,null,null,null,null,null,null,null,null,null,null,Valve,Valve,Action,"Valve Anti-Cheat enabled, Online PvP, Remote Play Together, Stereo Sound, PvP, Multi-player, Keyboard Only Option, Family Sharing, Custom Volume Controls, Color Alternatives, Shared/Split Screen PvP","linux, windows, mac"
50,Half-Life: Opposing Force,game,false,499.0,499.0,0.0,USD,null,22793,null,true,true,true,1999-11-01,0,499.0,499.0,22793,null,null,null,null,null,null,null,null,null,null,Gearbox Software,Valve,Action,"Valve Anti-Cheat enabled, Single-player, Adjustable Difficulty, Custom Volume Controls, Save Anytime, Playable without Timed Input, Family Sharing, Multi-player","windows, linux, mac"


In [0]:
display(reviews_app_df.limit(5))

appid,total_reviews,avg_votes_up,avg_votes_funny,avg_comment_count,avg_weighted_vote_score,avg_playtime_forever,pct_steam_purchase,pct_received_for_free,pct_early_access
1362650,17,1.1764705882352942,0.0,0.058823529411764705,0.5073456358823529,149.1764705882353,1.0,0.0,0.0
525640,100,5.26,0.23,0.22,0.5514197209000001,2874.67,1.0,0.01,0.74
962300,4,0.0,0.0,0.0,0.4953703725,0.0,1.0,0.0,0.0
617580,9,2.0,0.0,0.1111111111111111,0.5171290133333334,3744.6666666666665,1.0,0.1111111111111111,0.0
1633470,4,2.75,0.25,0.5,0.553060665,60.5,1.0,0.25,0.0


In [0]:
masterdata_df.coalesce(1) \
    .write.mode("overwrite") \
    .option("header", "true") \
    .csv("/Volumes/workspace/default/rawdata/steam/processed/masterdata_exports/masterdata/")


In [0]:
master_step1_df.coalesce(1) \
    .write.mode("overwrite") \
    .option("header", "true") \
    .csv("/Volumes/workspace/default/rawdata/steam/processed/masterdata_exports/master_step1_app_reviews/")


In [0]:
master_step2_df.coalesce(1) \
    .write.mode("overwrite") \
    .option("header", "true") \
    .csv("/Volumes/workspace/default/rawdata/steam/processed/masterdata_exports/master_step2_developers/")


In [0]:
master_step3_df.coalesce(1) \
    .write.mode("overwrite") \
    .option("header", "true") \
    .csv("/Volumes/workspace/default/rawdata/steam/processed/masterdata_exports/master_step3_publishers/")


In [0]:
master_step4_df.coalesce(1) \
    .write.mode("overwrite") \
    .option("header", "true") \
    .csv("/Volumes/workspace/default/rawdata/steam/processed/masterdata_exports/master_step4_genres/")


In [0]:
master_step5_df.coalesce(1) \
    .write.mode("overwrite") \
    .option("header", "true") \
    .csv("/Volumes/workspace/default/rawdata/steam/processed/masterdata_exports/master_step5_categories/")


In [0]:
dbutils.fs.ls("/Volumes/workspace/default/rawdata/steam/processed/masterdata_exports/")


[FileInfo(path='dbfs:/Volumes/workspace/default/rawdata/steam/processed/masterdata_exports/master_step1_app_reviews/', name='master_step1_app_reviews/', size=0, modificationTime=1769102302152),
 FileInfo(path='dbfs:/Volumes/workspace/default/rawdata/steam/processed/masterdata_exports/master_step2_developers/', name='master_step2_developers/', size=0, modificationTime=1769102302152),
 FileInfo(path='dbfs:/Volumes/workspace/default/rawdata/steam/processed/masterdata_exports/master_step3_publishers/', name='master_step3_publishers/', size=0, modificationTime=1769102302152),
 FileInfo(path='dbfs:/Volumes/workspace/default/rawdata/steam/processed/masterdata_exports/master_step4_genres/', name='master_step4_genres/', size=0, modificationTime=1769102302152),
 FileInfo(path='dbfs:/Volumes/workspace/default/rawdata/steam/processed/masterdata_exports/master_step5_categories/', name='master_step5_categories/', size=0, modificationTime=1769102302152),
 FileInfo(path='dbfs:/Volumes/workspace/defaul